In [1]:
 #using mystats env
import pandas as pd
import numpy as np
import pingouin as pg
import association_metrics as am
import scipy.stats as stats
from scipy.stats import chisquare
from scipy import stats
import pyreadstat
import statsmodels.api as sm
import warnings
import os
pd.options.display.float_format = '{:.2f}'.format

/Users/stephenwsong/anaconda3/envs/mystats/lib/python3.11/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.3, the latest is 0.5.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
#loading all the data
dfc=pd.read_csv('dfc.csv')
dfv=pd.read_csv('dfv.csv')
dfv_ColNames=pd.read_csv('dfv_ColNames.csv')
dfv_noDupe_ColNames=pd.read_csv('dfv_noDupe_ColNames.csv')
dfvnoDupe_recode=pd.read_csv('dfv_noDupe_recode.csv')
dfv_noDupe=pd.read_csv('dfv_noDupe.csv')
dfv_recode=pd.read_csv('dfv_recode.csv')
dfv_original=pd.read_csv('dfV_original.csv')
dfc_original=pd.read_csv('dfC_orignial.csv')

In [3]:
dfc.Case= dfc.Case.astype('category')
dfv.Case= dfv.Case.astype('category')

In [4]:
dfc.columns#errors in Publisher values

Index(['Case', 'CommCase', 'Platform', 'Publisher', 'VidTitle', 'CommentText',
       'CommLikes', 'CharacterNum', 'CommLength', 'Sentiment', 'Focus',
       'Alignment', 'MediaType'],
      dtype='object')

In [5]:
dfv.columns

Index(['Case', 'Platform', 'Publisher', 'VideoURL', 'PubDate', 'Likes',
       'Comments', 'Saves', 'Shots', 'Graphics', 'EditsStyle', 'Broll',
       'NatSound', 'Audio', 'Music', 'Narration', 'Humor', 'Offensive',
       'Sourcing', 'Fairness', 'Agenda', 'MediaIntegrity', 'Decorum',
       'Language', 'Topic', 'Presentation', 'ExternalVidSource', 'MediaType',
       'Animation', 'Filters'],
      dtype='object')

In [6]:
# merge dfc and selected columns from dfv_noDupe
dfc = pd.merge(dfc, dfv, on=['Case'], how='inner', indicator=True)

In [7]:
trendiness = dfv[['Humor', 'Animation', 'Filters', 'Music']]
pg.cronbach_alpha(data=trendiness)

(0.3537028623353021, array([0.148, 0.52 ]))

In [8]:
trendiness = dfv[['Shots', 'EditsStyle', 'Broll', 'NatSound', 'Audio', 'Music', 'Narration', 'Humor', 'Sourcing', 'Decorum','Language','Animation', 'Filters']]
pg.cronbach_alpha(data=trendiness)

(0.6993795035061768, array([0.616, 0.771]))

In [9]:
dfv['trendiness'] = dfv[['Shots', 'EditsStyle', 'Broll', 'NatSound', 'Audio', 'Music', 'Narration', 'Humor', 'Sourcing', 'Decorum','Language','Animation', 'Filters']].sum(axis=1)

In [10]:
dfv.Platform.value_counts()

Platform
YouTube Shorts     42
Instagram Reels    42
TikTok             42
Name: count, dtype: int64

RQ1: Does short-form video news lean more trendy or more traditional?

H1: Trendiness is more prevalent on TikTok compared to YouTube Shorts and Instagram Reels.

In [11]:
# Define the IV and DV variables
IV = 'Platform'
DV = ['trendiness']
DF = dfv
# Run the ANOVA
for i in DV:
    anova_result = pg.anova(data=DF, dv=i, between=IV)
    globals()[f"ANOVA_{i}"] = anova_result
    globals()[f"ANOVA_{i}"].insert(0, 'Variable', i)
    pairwise_comparisons = pg.pairwise_tests(data=DF, dv=i, between=IV, padjust='bonf')
    globals()[f"Pairwise_{i}"] = pairwise_comparisons
    globals()[f"Pairwise_{i}"].insert(0, 'Variable', i)
    mean_table = DF.groupby(IV)[i].mean().reset_index()
    std_table = DF.groupby(IV)[i].std().reset_index()
    mean_std_table = pd.merge(mean_table, std_table, on=IV)
    mean_std_table.columns = [i, 'Mean', 'Standard Deviation']
    mean_std_table = mean_std_table.rename(columns={i: 'Value'})
    globals()[f"mean_std_table_{i}"] = mean_std_table
    globals()[f"mean_std_table_{i}"].insert(0, 'Variable', i)
merged_anova = pd.concat([globals()[f"ANOVA_{i}"] for i in DV], axis=0)
merged_anova.reset_index(drop=True, inplace=True)
merged_pairwise = pd.concat([globals()[f"Pairwise_{i}"] for i in DV], axis=0)
merged_pairwise.reset_index(drop=True, inplace=True)
merged_mean_std_table = pd.concat([globals()[f"mean_std_table_{i}"] for i in DV], axis=0)
merged_mean_std_table.reset_index(drop=True, inplace=True)
# merged_anova.to_csv(IV+'/merged_anova.csv', index=False)
# merged_pairwise.to_csv(IV+'/merged_pairwise.csv', index=False)
# merged_mean_std_table.to_csv(IV+'/merged_mean_std.csv', index=False)
merged_anova

,Variable,Source,ddof1,ddof2,F,p-unc,np2
0,trendiness,Platform,2,123,0.94,0.39,0.02


In [12]:
merged_pairwise

,Variable,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,hedges
0,trendiness,Platform,Instagram Reels,TikTok,False,True,1.31,82.00,two-sided,0.19,0.58,bonf,0.479,0.28
1,trendiness,Platform,Instagram Reels,YouTube Shorts,False,True,0.91,82.00,two-sided,0.36,1.00,bonf,0.327,0.20
2,trendiness,Platform,TikTok,YouTube Shorts,False,True,-0.37,82.00,two-sided,0.71,1.00,bonf,0.242,-0.08


In [13]:
mean_std_table

,Variable,Value,Mean,Standard Deviation
0,trendiness,Instagram Reels,12.67,5.56
1,trendiness,TikTok,11.29,4.00
2,trendiness,YouTube Shorts,11.64,4.71



RQ2: Are people more positive towards a topic when news is presented more trendy or more traditional?

H2: People are more positive towards a topic when the presentation is trendy.


RQ3: When news is presented in traditional formats, do people engage less?

H3: Traditional presentations lead to less engagement online.


RQ4: Does post engagement occur more from positive, neutral, or negative comments?

H4: Negative comments reflect more engagements and interactions on posts.